##### CARGA INICIAL DE DEPENDENCIAS

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import shutil
import zipfile
from pathlib import Path

workdir = '/content'
os.chdir(workdir)

drive_base = '/content/drive/Shareddrives/NLP/entrega2_preprocessing+embeddings'
zip_files = [
    'output.zip',
    'bag_of_words.zip',
    'tf_idf.zip'
]

dest_root = Path(workdir)
dest_root.mkdir(parents=True, exist_ok=True)

for fname in zip_files:
    src_path = Path(drive_base) / fname
    local_zip = Path(workdir) / fname

    shutil.copy(str(src_path), str(local_zip))

    target_dir = dest_root / local_zip.stem
    target_dir.mkdir(parents=True, exist_ok=True)

    with zipfile.ZipFile(local_zip, 'r') as z:
        z.extractall(target_dir)

    print(f'✅ {fname} extraído en: {target_dir}')

!ls -lah "$dest_root"

✅ output.zip extraído en: /content/output
✅ bag_of_words.zip extraído en: /content/bag_of_words
✅ tf_idf.zip extraído en: /content/tf_idf
total 120M
drwxr-xr-x 1 root root 4.0K Jan 13 21:15 .
drwxr-xr-x 1 root root 4.0K Jan 13 21:10 ..
drwxr-xr-x 2 root root 4.0K Jan 13 21:15 bag_of_words
-rw------- 1 root root 394K Jan 13 21:29 bag_of_words.zip
drwxr-xr-x 4 root root 4.0K Dec  9 14:41 .config
drwx------ 5 root root 4.0K Jan 13 21:15 drive
drwxr-xr-x 3 root root 4.0K Jan 13 21:15 output
-rw------- 1 root root 119M Jan 13 21:29 output.zip
drwxr-xr-x 1 root root 4.0K Dec  9 14:42 sample_data
drwxr-xr-x 2 root root 4.0K Jan 13 21:15 tf_idf
-rw------- 1 root root 1.1M Jan 13 21:29 tf_idf.zip


Carga de dependencias principales

In [3]:
!pip install transformers datasets accelerate evaluate rouge_score sentencepiece huggingface_hub
!pip install -q -U torch peft trl
!pip install -U bitsandbytes

import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
import evaluate
import numpy as np
from datasets import load_dataset
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

### MODELOS BASADOS EN TRANSFORMERS

In [ ]:
import torch
import gc
import evaluate
import numpy as np
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from huggingface_hub import login

login(token="token")

# ==========================================
# 1. CARGA Y LIMPIEZA DE DATOS
# ==========================================
TEXT_COLUMN = "article"
SUMMARY_COLUMN = "summary"

print("Cargando dataset...")
raw_dataset = load_dataset("ELiRF/dacsa", "spanish")

NUM_TRAIN = 5000
NUM_VAL = 500

train_ds = raw_dataset["train"].shuffle(seed=0).select(range(min(NUM_TRAIN, len(raw_dataset["train"]))))
val_ds = raw_dataset["validation"].shuffle(seed=0).select(range(min(NUM_VAL, len(raw_dataset["validation"]))))

dataset = DatasetDict({"train": train_ds, "validation": val_ds})
print(f"Datos limpios: Train {len(dataset['train'])}, Val {len(dataset['validation'])}")


# ==========================================
# 2. CONFIGURACIÓN Y MÉTRICAS
# ==========================================
rouge = evaluate.load("rouge")

def compute_metrics_shared(eval_pred):
    predictions, labels = eval_pred
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Reemplazar -100 por pad token para decodificar
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Limpieza básica para Rouge
    decoded_preds = ["\n".join(pred.strip().split()) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.strip().split()) for label in decoded_labels]

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {k: round(v * 100, 4) for k, v in result.items()}

Cargando dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.17k [00:00<?, ?B/s]

spanish/train-00000-of-00014.parquet:   0%|          | 0.00/304M [00:00<?, ?B/s]

spanish/train-00001-of-00014.parquet:   0%|          | 0.00/303M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/1802919 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/104052 [00:00<?, ? examples/s]

Generating test.i split:   0%|          | 0/104052 [00:00<?, ? examples/s]

Generating test.ni split:   0%|          | 0/109626 [00:00<?, ? examples/s]

Datos limpios: Train 5000, Val 500


#### FINE-TUNING mT5 (base) de Google

In [ ]:
# ------------------------------------------
# 0. LIMPIEZA PREVIA
# ------------------------------------------
try:
    del model
    del trainer
    del tokenized_ds
    torch.cuda.empty_cache()
    gc.collect()
except:
    pass

MODEL_CHECKPOINT = "google/mt5-small"
OUTPUT_DIR = "./mt5-dacsa-finetuned"

print(f"mT5 Preparando: {MODEL_CHECKPOINT}")

# ==========================================
# 1. TOKENIZER
# ==========================================
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
MAX_INPUT = 512
MAX_TARGET = 128

def preprocess_mt5(examples):
    inputs = examples[TEXT_COLUMN]
    targets = examples[SUMMARY_COLUMN]

    model_inputs = tokenizer(inputs, max_length=MAX_INPUT, truncation=True)
    labels = tokenizer(text_target=targets, max_length=MAX_TARGET, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_ds = dataset.map(preprocess_mt5, batched=True, remove_columns=dataset["train"].column_names)

# ==========================================
# 2. ENTRENAMIENTO
# ==========================================
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

model.config.decoder_start_token_id = tokenizer.pad_token_id

args = Seq2SeqTrainingArguments(
    output_dir="./mt5-dacsa-finetuned",

    eval_strategy="epoch",
    save_strategy="epoch",

    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,
    eval_accumulation_steps=1,
    #gradient_checkpointing=True,

    learning_rate=2e-5,
    max_grad_norm=1.0,
    weight_decay=0.01,

    save_total_limit=2,
    num_train_epochs=6,

    predict_with_generate=True,
    generation_max_length=128,
    generation_num_beams=1,

    fp16=False,
    logging_steps=50,
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_shared
)

print("Iniciando entrenamiento...")
trainer.train()

mT5 Preparando: google/mt5-small


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/tmp/ipython-input-1205460259.py:78: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Iniciando entrenamiento...


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,4.264800,2.531432,14.821300,3.801600,12.368600,14.826000
2,3.568200,2.360449,18.408500,4.799900,14.393600,18.399600
3,3.454200,2.329967,25.591700,9.183700,20.224400,25.569600
4,3.308600,2.327950,25.847300,9.170100,20.584000,25.843500
5,3.283700,2.314652,25.805100,8.944700,20.416300,25.812500
6,3.263900,2.312581,26.106400,9.170000,20.648400,26.122200


TrainOutput(global_step=3750, training_loss=4.230730932617187, metrics={'train_runtime': 1879.9996, 'train_samples_per_second': 15.957, 'train_steps_per_second': 1.995, 'total_flos': 1.58624907264e+16, 'train_loss': 4.230730932617187, 'epoch': 6.0})

Guardar modelo

In [ ]:
output_drive_path = "/content/drive/Shareddrives/NLP/Entrega4/modelos/mt5-dacsa-finetuned"

trainer.save_model(output_drive_path)
tokenizer.save_pretrained(output_drive_path)

print(f"Modelo guardado en: {output_drive_path}.")

Modelo guardado en: /content/drive/Shareddrives/NLP/Entrega4/modelos/mt5-dacsa-finetuned.


Evaluation on ground truth

#### FINE-TUNING mBART (Large) de Facebook

In [ ]:
import torch
import gc
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)

# ------------------------------------------
# 0. LIMPIEZA PREVIA
# ------------------------------------------
try:
    del model
    del trainer
    del tokenized_ds
    torch.cuda.empty_cache()
    gc.collect()
except:
    pass

# ==========================================
# CONFIGURACIÓN mBART
# ==========================================
MODEL_CHECKPOINT = "facebook/mbart-large-50"
OUTPUT_DIR = "./mbart-dacsa-finetuned"

print(f"mBART Preparando: {MODEL_CHECKPOINT}")

# ==========================================
# 1. TOKENIZER (Configuración Español)
# ==========================================
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

# ¡CRÍTICO! Definir idiomas para mBART
tokenizer.src_lang = "es_XX"
tokenizer.tgt_lang = "es_XX"

MAX_INPUT = 512
MAX_TARGET = 128

def preprocess_mbart(examples):
    inputs = examples[TEXT_COLUMN]
    targets = examples[SUMMARY_COLUMN]

    # Tokenización
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT, truncation=True)
    labels = tokenizer(text_target=targets, max_length=MAX_TARGET, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Procesamos el dataset
tokenized_ds = dataset.map(preprocess_mbart, batched=True, remove_columns=dataset["train"].column_names)

# ==========================================
# 2. ENTRENAMIENTO
# ==========================================
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

# ¡CRÍTICO! Forzamos a mBART a empezar generando en español
model.config.forced_bos_token_id = tokenizer.lang_code_to_id["es_XX"]

args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,

    eval_strategy="epoch",
    save_strategy="epoch",

    # --- AJUSTE DE MEMORIA PARA mBART ---
    # mBART es más grande que mt5-small.
    # Bajamos batch a 4 y acumulamos 2 veces = Batch efectivo de 8 (igual que antes)
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,

    eval_accumulation_steps=1,       # Para evitar OOM en validación

    learning_rate=2e-5,
    max_grad_norm=1.0,
    weight_decay=0.01,

    save_total_limit=2,
    num_train_epochs=6,

    predict_with_generate=True,
    generation_max_length=128,
    generation_num_beams=1,

    fp16=True,

    logging_steps=50,
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_shared
)

print("Iniciando entrenamiento de mBART...")
trainer.train()

mBART Preparando: facebook/mbart-large-50


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/tmp/ipython-input-3712787654.py:103: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Iniciando entrenamiento de mBART...


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.075200,1.840636,28.518500,11.671100,22.437900,28.502800
2,2.661000,2.548765,20.923300,6.316400,16.706900,20.949800
3,1.377600,1.854803,28.242200,10.936300,22.413400,28.190500
4,1.093900,1.959458,27.463300,10.045000,21.730200,27.450700
5,0.888900,2.056684,26.856300,10.029800,21.654100,26.864800
6,0.818900,2.174418,27.315900,9.938900,21.693100,27.302500


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_bos_token_id': 250005}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3750, training_loss=1.3954578969319662, metrics={'train_runtime': 3246.306, 'train_samples_per_second': 9.241, 'train_steps_per_second': 1.155, 'total_flos': 3.247442264181965e+16, 'train_loss': 1.3954578969319662, 'epoch': 6.0})

In [ ]:
output_drive_path = "/content/drive/Shareddrives/NLP/Entrega4/modelos/mBART-dacsa-finetuned"

trainer.save_model(output_drive_path)
tokenizer.save_pretrained(output_drive_path)

print(f"Modelo BART guardado en: {output_drive_path}.")

Modelo BART guardado en: /content/drive/Shareddrives/NLP/Entrega4/modelos/mBART-dacsa-finetuned.


#### FINE-TUNING DEL MODELO CUANTIZADO (QLoRA)

In [ ]:
import os
import torch
import gc
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
from huggingface_hub import login

login(token="token")

# ------------------------------------------------------------------------
# 0. LIMPIEZA DE MEMORIA
# ------------------------------------------------------------------------
try:
    del model
    del trainer
    torch.cuda.empty_cache()
    gc.collect()
except:
    pass

# ------------------------------------------------------------------------
# 1. CONFIGURACIÓN GENERAL
# ------------------------------------------------------------------------
MODEL_NAME = "meta-llama/Meta-Llama-3-8B"
NEW_MODEL_NAME = "llama-3-8b-dacsa-finetuned"

TEXT_COLUMN = "article"
SUMMARY_COLUMN = "summary"

# ------------------------------------------------------------------------
# 2. CARGA DEL MODELO Y TOKENIZADOR
# ------------------------------------------------------------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

print(f"Cargando modelo base: {MODEL_NAME}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# ------------------------------------------------------------------------
# 3. PRE-PROCESAMIENTO DE DATOS
# ------------------------------------------------------------------------
def apply_chat_template(example):
    article_text = example[TEXT_COLUMN]
    summary_text = example[SUMMARY_COLUMN]

    prompt = f"""### Instrucción:
Resume el siguiente texto periodístico o transcripción.

### Texto de entrada:
{article_text}

### Resumen:
{summary_text}{tokenizer.eos_token}"""

    return {"formatted_text": prompt}

print("Procesando dataset...")
dataset = dataset.map(apply_chat_template)

# ------------------------------------------------------------------------
# 4. CONFIGURACIÓN LORA
# ------------------------------------------------------------------------
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.05,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj"]
)

# ------------------------------------------------------------------------
# 5. CONFIGURACIÓN DE ENTRENAMIENTO
# ------------------------------------------------------------------------
sft_config = SFTConfig(
    output_dir="./resultados_llama",
    dataset_text_field="formatted_text",
    max_steps=300,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=25,
    save_steps=100,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="none"
)

sft_config.max_seq_length = 1024
sft_config.packing = False

# ------------------------------------------------------------------------
# 6. INICIALIZACIÓN DEL TRAINER
# ------------------------------------------------------------------------
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    processing_class=tokenizer,
    args=sft_config,
)

# Ajuste de estabilidad para evitar errores de gradiente en T4
for name, param in model.named_parameters():
    if ".lora_" in name or ".modules_to_save" in name:
        param.data = param.data.to(torch.float32)
        param.requires_grad = True

print("\nIniciando entrenamiento ...")
trainer.train()

# ==============================================================================
# GUARDADO (SOLO ADAPTADORES LORA)
# ==============================================================================
import os
from google.colab import drive

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

ADAPTER_PATH = "/content/drive/MyDrive/NLP_Entrega4_modelos/llama3-dacsa-adapters-only"

print(f"Guardar adaptadores: {ADAPTER_PATH}")

trainer.model.save_pretrained(ADAPTER_PATH)
tokenizer.save_pretrained(ADAPTER_PATH)

print("Adaptadores guardados")

`torch_dtype` is deprecated! Use `dtype` instead!


Cargando modelo base: meta-llama/Meta-Llama-3-8B


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Procesando dataset...


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

...

Truncating train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 128001}.



Iniciando entrenamiento ...


Step,Training Loss
25,1.975200
50,1.754000
75,1.941400
100,1.700000
125,1.923800
150,1.738700
175,1.919700
200,1.715600
225,1.904300
250,1.724500


Guardar adaptadores: /content/drive/MyDrive/NLP_Entrega4_modelos/llama3-dacsa-adapters-only
Adaptadores guardados


#### Sanity check para transcripciones problemáticas

In [ ]:
# === 1. Cargar archivo ===
file_path = "/content/drive/Shareddrives/NLP/chunks_conTags/ABOGADOS TUMBAN los BULOS sobre EXTRANJERIA_conTags.txt"
df = pd.read_csv(file_path)

# Verificar columna
if 'text' not in df.columns:
    raise ValueError("El archivo debe contener una columna llamada 'text'.")

print(f"Filas: {len(df)} | Columnas: {df.columns.tolist()}")

Filas: 633 | Columnas: ['video_id', 'chunk_id', 'text', 'sentimiento']


In [ ]:
# === 1. Cargar archivo ===

transcription_names = ['ABOGADOS TUMBAN los BULOS sobre EXTRANJERIA_conTags.txt',
                       'Analizamos el ACUERDO COMERCIAL entre la UE y EEUU_conTags.txt',
                       'ARANCELES_ TRUMP REVIENTA SU PROPIA BOLSA_conTags',
                       'CARCEL por PROTESTAR(con Francho AIJON)_conTags',
                       'Caso MONTORO_ COMO FUNCIONAN las CLOACAS_conTags',
                       'CAYENDO por su propo Peso_conTags',
                       'COMUNICACION POLITICA y HEGEMONIA_conTags',
                       'Cuanta PRESION puede AGUANTAR Israel_conTags',
                       'EL BRAZO ARMADO de TRUMP_conTags',
                       'El SINTOMA de un ENORME PROBLEMA POLITICO_conTags']


file_path = "/content/drive/Shareddrives/NLP/chunks_conTags/ABOGADOS TUMBAN los BULOS sobre EXTRANJERIA_conTags.txt"
file_path = "/content/drive/Shareddrives/NLP/chunks_conTags/Analizamos el ACUERDO COMERCIAL entre la UE y EEUU_conTags.txt"

df = pd.read_csv(file_path)

# Verificar columna
if 'text' not in df.columns:
    raise ValueError("El archivo debe contener una columna llamada 'text'.")

print(f"Filas: {len(df)} | Columnas: {df.columns.tolist()}")

Filas: 418 | Columnas: ['video_id', 'chunk_id', 'text', 'sentimiento']


In [ ]:
transcription_names = ["ABOGADOS TUMBAN los BULOS sobre EXTRANJERIA_conTags.txt",
                       "Analizamos el ACUERDO COMERCIAL entre la UE y EEUU_conTags.txt",
                       "ARANCELES_ TRUMP REVIENTA SU PROPIA BOLSA_conTags.txt",
                       "CARCEL por PROTESTAR(con Francho AIJON)_conTags.txt",
                       "Caso MONTORO_ COMO FUNCIONAN las CLOACAS_conTags.txt",
                       "CAYENDO por su propo Peso_conTags.txt",
                       "COMUNICACION POLITICA y HEGEMONIA_conTags.txt",
                       "Cuanta PRESION puede AGUANTAR Israel_conTags.txt",
                       "EL BRAZO ARMADO de TRUMP_conTags.txt",
                       "El SINTOMA de un ENORME PROBLEMA POLITICO_conTags.txt"]

for transcription_name in transcription_names:
  file_path = "/content/drive/Shareddrives/NLP/chunks_conTags/" + transcription_name
  df = pd.read_csv(file_path)
  expected_cols = None
  problematic_rows = []

  with open(file_path, 'r', encoding='utf-8') as f:
      reader = csv.reader(f)
      for i, row in enumerate(reader):
          if i == 0:
              expected_cols = len(row)
          else:
              if len(row) != expected_cols:
                  problematic_rows.append((i+1, len(row), row))

  print(f"Sanity check for: {transcription_name}")
  print(f"Número esperado de columnas: {expected_cols}")
  print(f"Total filas problemáticas: {len(problematic_rows)}")

  # Mostrar las primeras 10 filas problemáticas
  for r in problematic_rows[:10]:
      print(f"Línea {r[0]} -> columnas: {r[1]} | contenido: {r[2]}")


Sanity check for: ABOGADOS TUMBAN los BULOS sobre EXTRANJERIA_conTags.txt
Número esperado de columnas: 4
Total filas problemáticas: 0
Sanity check for: Analizamos el ACUERDO COMERCIAL entre la UE y EEUU_conTags.txt
Número esperado de columnas: 4
Total filas problemáticas: 0
Sanity check for: ARANCELES_ TRUMP REVIENTA SU PROPIA BOLSA_conTags.txt
Número esperado de columnas: 4
Total filas problemáticas: 0
Sanity check for: CARCEL por PROTESTAR(con Francho AIJON)_conTags.txt
Número esperado de columnas: 4
Total filas problemáticas: 0
Sanity check for: Caso MONTORO_ COMO FUNCIONAN las CLOACAS_conTags.txt
Número esperado de columnas: 4
Total filas problemáticas: 0
Sanity check for: CAYENDO por su propo Peso_conTags.txt
Número esperado de columnas: 4
Total filas problemáticas: 0
Sanity check for: COMUNICACION POLITICA y HEGEMONIA_conTags.txt
Número esperado de columnas: 4
Total filas problemáticas: 0
Sanity check for: Cuanta PRESION puede AGUANTAR Israel_conTags.txt
Número esperado de column

## TESTEAR MODELOS CON GROUND TRUTH

Testeo simple: Límite de lectura de la transcripción original y resumen reducido

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from google.colab import drive

# ==============================================================================
# 2. INICIALIZACIÓN
# ==============================================================================
def initialize_test(test_model):
  RUTAS_MODELOS = {
      "mT5": "/content/drive/Shareddrives/NLP/Entrega4/modelos/mt5-dacsa-finetuned",
      "mBART": "/content/drive/Shareddrives/NLP/Entrega4/modelos/mBART-dacsa-finetuned"
  }

  if not os.path.exists('/content/drive'):
      drive.mount('/content/drive')

  transcript_folder = "/content/drive/Shareddrives/NLP/transcription_summaries/transcriptions"
  summary_gt_folder = "/content/drive/Shareddrives/NLP/transcription_summaries/summaries"

  selected_path = RUTAS_MODELOS[test_model]
  print(f"Seleccionado: {selected_path}")
  print(f"Cargando desde: {selected_path}")

  device = "cuda" if torch.cuda.is_available() else "cpu"
  print(f"Dispositivo: {device}")

  try:
      tokenizer = AutoTokenizer.from_pretrained(selected_path)
      model = AutoModelForSeq2SeqLM.from_pretrained(selected_path).to(device)

      # --- CONFIGURACIÓN ESPECÍFICA PARA mBART ---
      if test_model == "mBART":
          print("Aplicando configuración extra para mBART (src_lang=es_XX)...")
          tokenizer.src_lang = "es_XX"

      print("Modelo cargado correctamente.")

      return model, tokenizer, device, transcript_folder, summary_gt_folder

  except Exception as e:
      print(f"Error cargando el modelo: {e}")
      raise e

# ==============================================================================
# 3. FUNCIÓN DE RESUMEN
# ==============================================================================
def summarize(model, tokenizer, text, test_model, device):
    # 1. Preprocesar
    inputs = tokenizer(
        text,
        max_length=1024,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    # 2. Preparar argumentos dinámicos para .generate()
    gen_kwargs = {
        "max_length": 300,
        "min_length": 40,
        "num_beams": 2,
        "early_stopping": True,
        "no_repeat_ngram_size": 2
    }

    # --- LÓGICA CONDICIONAL ---
    # Si es mBART, forzamos el idioma de salida. Si es mT5, no hacemos nada extra.
    if test_model == "mBART":
        gen_kwargs["forced_bos_token_id"] = tokenizer.lang_code_to_id["es_XX"]

    # 3. Generar
    summary_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        **gen_kwargs  # Desempaquetamos los argumentos (incluido el forced_bos si existe)
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# ==============================================================================
# 4. LECTURA DE ARCHIVOS
# ==============================================================================
def read_txt(path):
    with open(path, "r", encoding="utf-8") as f:
        return f.read().strip()

def compare_summaries(test_model):
    model, tokenizer, device, transcript_folder, summary_gt_folder = initialize_test(test_model)

    if os.path.exists(transcript_folder):
        transcript_files = sorted([f for f in os.listdir(transcript_folder) if f.endswith(".txt")])
        print(f"\nArchivos encontrados: {len(transcript_files)}")
        print(f"Generando resúmenes con: {test_model}\n")

        count = 0
        LIMIT = 4

        for transcript_file in transcript_files:
            if count >= LIMIT: break

            transcript_path = os.path.join(transcript_folder, transcript_file)
            base_name = transcript_file.replace(".txt", "")
            summary_file = base_name + "_summary.txt"
            summary_path = os.path.join(summary_gt_folder, summary_file)

            # Leer textos
            transcript = read_txt(transcript_path)
            try:
                summary_gt = read_txt(summary_path)
            except FileNotFoundError:
                summary_gt = "<No existe ground truth>"

            # Generar resumen
            print(f"Procesando: {transcript_file}...")
            summary_pred = summarize(model, tokenizer, transcript, test_model, device)

            # Imprimir Resultados
            print("-" * 60)
            print(f"Archivo: {transcript_file}")
            print("-" * 60)
            print("🔵 Ground truth:")
            print(summary_gt)
            print("-" * 30)
            print(f"🟢 Resumen generado ({test_model}):")
            print(summary_pred)
            print("=" * 60 + "\n")

            count += 1
    else:
        print(f"Error: Carpeta no encontrada: {transcript_folder}")

#### MODELO mT5

In [ ]:
compare_summaries("mT5")

Seleccionado: /content/drive/Shareddrives/NLP/Entrega4/modelos/mt5-dacsa-finetuned
Cargando desde: /content/drive/Shareddrives/NLP/Entrega4/modelos/mt5-dacsa-finetuned
Dispositivo: cuda


The tokenizer you are loading from '/content/drive/Shareddrives/NLP/Entrega4/modelos/mt5-dacsa-finetuned' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Modelo cargado correctamente.

Archivos encontrados: 10
Generando resúmenes con: mT5

Procesando: ABOGADOS TUMBAN los BULOS sobre EXTRANJERÍA.txt...
------------------------------------------------------------
Archivo: ABOGADOS TUMBAN los BULOS sobre EXTRANJERÍA.txt
------------------------------------------------------------
🔵 Ground truth:
Este programa desmonta los bulos racistas sobre la extranjería en España, explicando el marco legal real. Se aclara que la Ley de Extranjería y el Código Penal sí permiten la expulsión de extranjeros en situación irregular o que cometan delitos, desmintiendo la narrativa de impunidad. Se critica la confusión interesada entre el gobierno, la administración y el poder judicial, y se denuncia que el discurso de la ultradereja sobre "deportaciones masivas" es una apuesta por la arbitrariedad premoderna y el "derecho penal del enemigo", que elimina las garantías procesales individuales. El análisis concluye que este racismo, basado en falsedades, busc

#### MODELO mBART

In [ ]:
compare_summaries("mBART")

Seleccionado: /content/drive/Shareddrives/NLP/Entrega4/modelos/mBART-dacsa-finetuned
Cargando desde: /content/drive/Shareddrives/NLP/Entrega4/modelos/mBART-dacsa-finetuned
Dispositivo: cuda


The tokenizer you are loading from '/content/drive/Shareddrives/NLP/Entrega4/modelos/mBART-dacsa-finetuned' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Aplicando configuración extra para mBART (src_lang=es_XX)...
Modelo cargado correctamente.

Archivos encontrados: 10
Generando resúmenes con: mBART

Procesando: ABOGADOS TUMBAN los BULOS sobre EXTRANJERÍA.txt...
------------------------------------------------------------
Archivo: ABOGADOS TUMBAN los BULOS sobre EXTRANJERÍA.txt
------------------------------------------------------------
🔵 Ground truth:
Este programa desmonta los bulos racistas sobre la extranjería en España, explicando el marco legal real. Se aclara que la Ley de Extranjería y el Código Penal sí permiten la expulsión de extranjeros en situación irregular o que cometan delitos, desmintiendo la narrativa de impunidad. Se critica la confusión interesada entre el gobierno, la administración y el poder judicial, y se denuncia que el discurso de la ultradereja sobre "deportaciones masivas" es una apuesta por la arbitrariedad premoderna y el "derecho penal del enemigo", que elimina las garantías procesales individuales. El

#### **Inferencia basada en chunking**: Resumenes recursivos de bloques más reducidos del texto original (Lógica Map-Reduce)

In [ ]:
import os
import torch
import evaluate
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from google.colab import drive

# ==============================================================================
# 1. INICIALIZACIÓN
# ==============================================================================
def initialize_test(test_model):
    # Rutas (Asegúrate que coinciden con tu Drive)
    RUTAS_MODELOS = {
        "mT5": "/content/drive/Shareddrives/NLP/Entrega4/modelos/mt5-dacsa-finetuned",
        "mBART": "/content/drive/Shareddrives/NLP/Entrega4/modelos/mBART-dacsa-finetuned"
    }

    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')

    transcript_folder = "/content/drive/Shareddrives/NLP/transcription_summaries/transcriptions"
    summary_gt_folder = "/content/drive/Shareddrives/NLP/transcription_summaries/summaries"

    selected_path = RUTAS_MODELOS.get(test_model)

    # Verificación básica
    if not selected_path:
        raise ValueError(f"Modelo '{test_model}' no definido en RUTAS_MODELOS")

    print(f"\nInicializando configuración: {test_model}")
    print(f"Cargando desde: {selected_path}")

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Dispositivo: {device}")

    try:
        tokenizer = AutoTokenizer.from_pretrained(selected_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(selected_path).to(device)

        # Configuración específica para mBART
        if test_model == "mBART":
            print("Aplicando configuración extra para mBART (src_lang=es_XX)...")
            tokenizer.src_lang = "es_XX"

        print("Modelo cargado correctamente.")
        return model, tokenizer, device, transcript_folder, summary_gt_folder

    except Exception as e:
        print(f"Error cargando el modelo: {e}")
        raise e

# ==============================================================================
# 2. GENERACIÓN DE TEXTO (BASE)
# ==============================================================================
def generate_text(text, max_len, min_len, penalty, model, tokenizer, device, test_model):
    """Genera texto manejando las diferencias entre mT5 y mBART"""

    inputs = tokenizer(
        text,
        max_length=1024,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    gen_kwargs = {
        "max_length": max_len,
        "min_length": min_len,
        "length_penalty": penalty,
        "num_beams": 4,
        "no_repeat_ngram_size": 3,
        "early_stopping": True
    }

    # mBART necesita el token de inicio forzado
    if test_model == "mBART":
        gen_kwargs["forced_bos_token_id"] = tokenizer.lang_code_to_id["es_XX"]

    summary_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        **gen_kwargs
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# ==============================================================================
# 3. LÓGICA RECURSIVA (MAP-REDUCE)
# ==============================================================================
def recursive_summarize(text, model, tokenizer, device, test_model):
    """Divide, resume partes y sintetiza el final"""

    # 1. Chunking
    all_tokens = tokenizer.encode(text, add_special_tokens=False)
    chunk_size = 1024
    chunks = []
    for i in range(0, len(all_tokens), chunk_size):
        chunk_ids = all_tokens[i : i + chunk_size]
        chunks.append(tokenizer.decode(chunk_ids, skip_special_tokens=True))

    # 2. Map (Resúmenes parciales)
    partial_summaries = []
    for chunk in chunks:
        summary = generate_text(chunk, 150, 50, 1, model, tokenizer, device, test_model)
        partial_summaries.append(summary)

    intermediate_text = " ".join(partial_summaries)

    # 3. Reduce (Resumen final)
    final_summary = generate_text(intermediate_text, 500, 150, 1, model, tokenizer, device, test_model)

    return final_summary

# ==============================================================================
# 4. EJECUCIÓN Y VISUALIZACIÓN
# ==============================================================================
def read_txt(path):
    with open(path, "r", encoding="utf-8") as f: return f.read().strip()

def run_evaluation(test_model_name):
    model, tokenizer, device, transcript_folder, summary_gt_folder = initialize_test(test_model_name)

    print("Cargando librería ROUGE...")
    rouge_metric = evaluate.load("rouge")

    if os.path.exists(transcript_folder):
        transcript_files = sorted([f for f in os.listdir(transcript_folder) if f.endswith(".txt")])
        print(f"\nArchivos encontrados: {len(transcript_files)}")
        print(f"Generando con: {test_model_name}\n")

        # Listas para calcular la media global al final
        all_preds = []
        all_gts = []

        count = 0
        LIMIT = 3

        for transcript_file in transcript_files:
            if count >= LIMIT: break

            # Rutas
            t_path = os.path.join(transcript_folder, transcript_file)
            s_path = os.path.join(summary_gt_folder, transcript_file.replace(".txt", "_summary.txt"))

            # Leer
            transcript = read_txt(t_path)
            try:
                summary_gt = read_txt(s_path)
                has_gt = True
            except:
                summary_gt = "<No existe ground truth>"
                has_gt = False

            print(f"Procesando: {transcript_file}...")

            # Generar
            summary_pred = recursive_summarize(transcript, model, tokenizer, device, test_model_name)

            # Calcular métricas individuales
            scores = {}
            if has_gt:
                scores = rouge_metric.compute(predictions=[summary_pred], references=[summary_gt])
                all_preds.append(summary_pred)
                all_gts.append(summary_gt)

            # --- VISUALIZACIÓN ---
            print("-" * 60)
            print(f"Archivo: {transcript_file}")
            print("-" * 60)
            print("🔵 Ground truth:")
            print(summary_gt)
            print("-" * 30)
            print(f"🟢 Resumen generado ({test_model_name}):")
            print(summary_pred)

            if has_gt:
                print("-" * 30)
                print("MÉTRICAS ROUGE:")
                print(f"   R1: {scores['rouge1']:.4f} | R2: {scores['rouge2']:.4f} | RL: {scores['rougeL']:.4f}")
            else:
                print("-" * 30)
                print("Sin métricas (Falta GT)")

            print("=" * 60 + "\n")
            count += 1

        # --- RESULTADO FINAL GLOBAL ---
        if len(all_preds) > 0:
            print("\n" + "=" * 30)
            print(f" RESULTADO GLOBAL PROMEDIO")
            print("=" * 30)
            final_scores = rouge_metric.compute(predictions=all_preds, references=all_gts)
            print(f"ROUGE-1: {final_scores['rouge1']:.4f}")
            print(f"ROUGE-2: {final_scores['rouge2']:.4f}")
            print(f"ROUGE-L: {final_scores['rougeL']:.4f}")

    else:
        print("Error: Carpeta no encontrada.")

In [ ]:
run_evaluation("mT5")


Inicializando configuración: mT5
Cargando desde: /content/drive/Shareddrives/NLP/Entrega4/modelos/mt5-dacsa-finetuned
Dispositivo: cuda


The tokenizer you are loading from '/content/drive/Shareddrives/NLP/Entrega4/modelos/mt5-dacsa-finetuned' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Modelo cargado correctamente.
Cargando librería ROUGE...

Archivos encontrados: 10
Generando con: mT5

Procesando: ABOGADOS TUMBAN los BULOS sobre EXTRANJERÍA.txt...
------------------------------------------------------------
Archivo: ABOGADOS TUMBAN los BULOS sobre EXTRANJERÍA.txt
------------------------------------------------------------
🔵 Ground truth:
Este programa desmonta los bulos racistas sobre la extranjería en España, explicando el marco legal real. Se aclara que la Ley de Extranjería y el Código Penal sí permiten la expulsión de extranjeros en situación irregular o que cometan delitos, desmintiendo la narrativa de impunidad. Se critica la confusión interesada entre el gobierno, la administración y el poder judicial, y se denuncia que el discurso de la ultradereja sobre "deportaciones masivas" es una apuesta por la arbitrariedad premoderna y el "derecho penal del enemigo", que elimina las garantías procesales individuales. El análisis concluye que este racismo, basado en

In [ ]:
run_evaluation("mBART")


Inicializando configuración: mBART
Cargando desde: /content/drive/Shareddrives/NLP/Entrega4/modelos/mBART-dacsa-finetuned
Dispositivo: cuda


The tokenizer you are loading from '/content/drive/Shareddrives/NLP/Entrega4/modelos/mBART-dacsa-finetuned' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Aplicando configuración extra para mBART (src_lang=es_XX)...
Modelo cargado correctamente.
Cargando librería ROUGE...


Token indices sequence length is longer than the specified maximum sequence length for this model (20579 > 1024). Running this sequence through the model will result in indexing errors



Archivos encontrados: 10
Generando con: mBART

Procesando: ABOGADOS TUMBAN los BULOS sobre EXTRANJERÍA.txt...
------------------------------------------------------------
Archivo: ABOGADOS TUMBAN los BULOS sobre EXTRANJERÍA.txt
------------------------------------------------------------
🔵 Ground truth:
Este programa desmonta los bulos racistas sobre la extranjería en España, explicando el marco legal real. Se aclara que la Ley de Extranjería y el Código Penal sí permiten la expulsión de extranjeros en situación irregular o que cometan delitos, desmintiendo la narrativa de impunidad. Se critica la confusión interesada entre el gobierno, la administración y el poder judicial, y se denuncia que el discurso de la ultradereja sobre "deportaciones masivas" es una apuesta por la arbitrariedad premoderna y el "derecho penal del enemigo", que elimina las garantías procesales individuales. El análisis concluye que este racismo, basado en falsedades, busca crear un chivo expiatorio y oculta l

#### LLAMA 3 (8B)

Al haber guardado en Drive únicamente los daptadores de LoRA fine-tuneados previamente (por cuestiones de límites en el almacenamiento) en este paso es necesario realizar la fusión entre el modelo cuantizado preentrenado y los adaptadores para poder realizar la inferencia.

In [2]:
import os
import torch
import evaluate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from google.colab import drive

# ==============================================================================
# 1. INICIALIZACIÓN (MODO ADAPTADORES)
# ==============================================================================
def initialize_llama_test():

    # Rutas de adaptadores y modelo base
    ADAPTER_PATH = "/content/drive/MyDrive/NLP_Entrega4_modelos/llama3-dacsa-adapters-only"
    BASE_MODEL_NAME = "meta-llama/Meta-Llama-3-8B"

    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')

    transcript_folder = "/content/drive/Shareddrives/NLP/transcription_summaries/transcriptions"
    summary_gt_folder = "/content/drive/Shareddrives/NLP/transcription_summaries/summaries"

    print(f"\nInicializando Llama 3 (Base + LoRA)...")

    # Configuración 4-bits
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False,
    )

    try:
        # Cargar Tokenizer
        tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        # Cargar Modelo Base
        print(f"Cargando modelo base: {BASE_MODEL_NAME}")
        base_model = AutoModelForCausalLM.from_pretrained(
            BASE_MODEL_NAME,
            quantization_config=bnb_config,
            device_map="auto",
            torch_dtype=torch.float16
        )

        # Inyectar Adaptadores
        print(f"Fusionando adaptadores desde: {ADAPTER_PATH}")
        model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)

        print("Modelo ensamblado correctamente.")
        return model, tokenizer, transcript_folder, summary_gt_folder

    except Exception as e:
        print(f"Error cargando el modelo: {e}")
        raise e

# ==============================================================================
# 2. GENERACIÓN (ADAPTADA A LLAMA 3)
# ==============================================================================
# ==============================================================================
# 2. GENERACIÓN MEJORADA (BEAM SEARCH + FORZAR LONGITUD)
# ==============================================================================
def generate_llama_response(text_chunk, model, tokenizer):

    # 1. Prompt explícito pidiendo extensión
    prompt = f"""### Instrucción:
Genera un resumen extenso y detallado del siguiente texto. Asegúrate de capturar todos los puntos principales y desarrollar las ideas.

### Texto de entrada:
{text_chunk}

### Resumen:
"""

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            # --- PARÁMETROS CRÍTICOS DE LONGITUD ---
            max_new_tokens=300,      # Damos mucho espacio
            min_new_tokens=100,      # OBLIGAMOS a escribir al menos 100 tokens nuevos

            # --- ESTRATEGIA: BEAM SEARCH (Mejor para resúmenes que 'sample') ---
            num_beams=4,             # Busca las 4 mejores rutas (reduce alucinaciones)
            do_sample=False,         # Desactivamos aleatoriedad para que length_penalty funcione
            length_penalty=1.5,      # > 1.0 fuerza al modelo a preferir frases largas (ahora sí funciona)

            # --- CONTROL ---
            repetition_penalty=1.2,  # Evita que se repita para llenar espacio
            no_repeat_ngram_size=3,  # Evita repetir frases de 3 palabras exactas

            eos_token_id=terminators,
            pad_token_id=tokenizer.pad_token_id,
        )

    # Limpieza
    response = outputs[0][inputs['input_ids'].shape[-1]:]
    decoded_output = tokenizer.decode(response, skip_special_tokens=True)

    return decoded_output.strip()

# ==============================================================================
# 3. LÓGICA RECURSIVA (CHUNKING)
# ==============================================================================
def recursive_summarize_llama(text, model, tokenizer):
    chunk_size = 2048
    tokens = tokenizer.encode(text, add_special_tokens=False)

    if len(tokens) < 6000:
        return generate_llama_response(text, model, tokenizer)

    chunks = []
    for i in range(0, len(tokens), chunk_size):
        chunk_ids = tokens[i : i + chunk_size]
        chunks.append(tokenizer.decode(chunk_ids, skip_special_tokens=True))

    partial_summaries = []
    for chunk in chunks:
        partial_summaries.append(generate_llama_response(chunk, model, tokenizer))

    intermediate_text = "\n".join(partial_summaries)
    return generate_llama_response(intermediate_text, model, tokenizer)

# ==============================================================================
# 4. EJECUCIÓN
# ==============================================================================
def read_txt(path):
    with open(path, "r", encoding="utf-8") as f: return f.read().strip()

def run_llama_evaluation():
    model, tokenizer, transcript_folder, summary_gt_folder = initialize_llama_test()
    rouge_metric = evaluate.load("rouge")

    if os.path.exists(transcript_folder):
        transcript_files = sorted([f for f in os.listdir(transcript_folder) if f.endswith(".txt")])
        print(f"\nArchivos encontrados: {len(transcript_files)}")

        all_preds = []
        all_gts = []
        count = 0
        LIMIT = 2

        for transcript_file in transcript_files:
            if count >= LIMIT: break

            t_path = os.path.join(transcript_folder, transcript_file)
            s_path = os.path.join(summary_gt_folder, transcript_file.replace(".txt", "_summary.txt"))

            transcript = read_txt(t_path)
            try:
                summary_gt = read_txt(s_path)
                has_gt = True
            except:
                summary_gt = "<No existe ground truth>"
                has_gt = False

            print(f"Procesando: {transcript_file}")
            summary_pred = recursive_summarize_llama(transcript, model, tokenizer)

            if has_gt:
                scores = rouge_metric.compute(predictions=[summary_pred], references=[summary_gt])
                all_preds.append(summary_pred)
                all_gts.append(summary_gt)

            print("-" * 60)
            print(f"Archivo: {transcript_file}")
            print("-" * 60)
            print("🔵 Ground truth:")
            print(summary_gt)
            print("-" * 30)
            print(f"🟢 Resumen generado (Llama 3):")
            print(summary_pred)

            if has_gt:
                print("-" * 30)
                print("MÉTRICAS ROUGE:")
                print(f"   R1: {scores['rouge1']:.4f} | R2: {scores['rouge2']:.4f} | RL: {scores['rougeL']:.4f}")
            else:
                print("-" * 30)
                print("Sin métricas (Falta GT)")

            print("=" * 60 + "\n")
            count += 1

        if len(all_preds) > 0:
            print("\n" + "=" * 30)
            print(f" RESULTADO GLOBAL PROMEDIO")
            print("=" * 30)
            final_scores = rouge_metric.compute(predictions=all_preds, references=all_gts)
            print(f"ROUGE-1: {final_scores['rouge1']:.4f}")
            print(f"ROUGE-2: {final_scores['rouge2']:.4f}")
            print(f"ROUGE-L: {final_scores['rougeL']:.4f}")
    else:
        print("Error: Carpeta no encontrada.")

In [3]:
run_llama_evaluation()


Inicializando Llama 3 (Base + LoRA)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Cargando modelo base: meta-llama/Meta-Llama-3-8B


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Fusionando adaptadores desde: /content/drive/MyDrive/NLP_Entrega4_modelos/llama3-dacsa-adapters-only
Modelo ensamblado correctamente.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Archivos encontrados: 10
Procesando: ABOGADOS TUMBAN los BULOS sobre EXTRANJERÍA.txt
------------------------------------------------------------
Archivo: ABOGADOS TUMBAN los BULOS sobre EXTRANJERÍA.txt
------------------------------------------------------------
🔵 Ground truth:
Este programa desmonta los bulos racistas sobre la extranjería en España, explicando el marco legal real. Se aclara que la Ley de Extranjería y el Código Penal sí permiten la expulsión de extranjeros en situación irregular o que cometan delitos, desmintiendo la narrativa de impunidad. Se critica la confusión interesada entre el gobierno, la administración y el poder judicial, y se denuncia que el discurso de la ultradereja sobre "deportaciones masivas" es una apuesta por la arbitrariedad premoderna y el "derecho penal del enemigo", que elimina las garantías procesales individuales. El análisis concluye que este racismo, basado en falsedades, busca crear un chivo expiatorio y oculta la explotación económica d